In [1]:
import pylab as pl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
# Load Data:
# ETFS:
USO = pd.read_csv("USO.csv", index_col=0)
UNG = pd.read_csv("UNG.csv", index_col=0)
USO = USO.dropna()
UNG = UNG.dropna()

# Futures:
WTI = pd.read_csv("WTI.csv", index_col=0)
NG = pd.read_csv("NG.csv", index_col=0)
WTI = WTI.dropna()
NG = NG.dropna()

USO = USO.join(WTI["Open Interest"])
UNG = UNG.join(NG["Open Interest"])
USO = USO.dropna()
UNG = UNG.dropna()


In [2]:
def getInterest(x):
    if x[-1] == 'k':
        return float(x.split('k')[0]) * 1000
    return float(x)

USO["Open Interest"] = USO['Open Interest'].apply(getInterest)
UNG["Open Interest"] = UNG['Open Interest'].apply(getInterest)


In [4]:
#appends price shift and target variables to the DF
def calculatePriceShift(df):
    prices = df['Last Px'].tolist()[1:] + [0]
    df['Price Shift'] = prices - df['Last Px']
    df['Target'] = df['Price Shift'].apply(lambda x: 1 if x>0 else 0 )
    return df

USO = calculatePriceShift(USO)
UNG = calculatePriceShift(USO)

openi = [0] + USO["Open Interest"].tolist()
openi = openi[:-1]

USO['OI Shift'] = USO["Open Interest"] - openi
USO['OI Change'] = USO['OI Shift'].apply(lambda x: 1 if x > 0 else 0)


,Last Px,P/C,Open Interest,Price Shift,Target
Date,,,,,
11/6/18,13.1800,2.1956,377866.0,0.1200,1
11/5/18,13.3000,0.8371,399347.0,0.0100,1
11/2/18,13.3100,0.6289,424118.0,0.1500,1
11/1/18,13.4600,1.7868,435959.0,0.3300,1
10/31/18,13.7900,0.6765,432487.0,0.2400,1
10/30/18,14.0300,0.4885,443231.0,0.0800,1
10/29/18,14.1100,0.4266,454348.0,0.2200,1
10/26/18,14.3300,0.8873,459476.0,-0.1200,0
10/25/18,14.2100,1.5302,463771.0,-0.1600,0


In [49]:
model = SVC()
train = USO.iloc[:2000]
test = USO.iloc[2000:]

train = UNG.iloc[:2000]
test = UNG.iloc[2000:]

In [50]:
model.fit(train['P/C'].values.reshape(-1,1), train['Target'])
results = model.predict(test['P/C'].values.reshape(-1,1))
test['predicted'] = results

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
accuracy_score(test['Target'], results)


In [52]:
matthews_corrcoef(test['Target'], results)


/Users/joedevito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
roc_auc_score(test['Target'], results)


0.5602294455066922